## Web Scraping de la Flota Europea con Selenium

Se realiza Web Scraping para capturar datos desde dos funtes y estrategias distintas:

Por un lado, este utiliza Selenium para automatizar la extracción de datos desde el sitio web de Fleet Europa de la Unión Europea. El script automatiza la búsqueda de todos los barcos registrados, navega a través de las páginas de resultados y guarda los datos en un archivo CSV. Para esto ejecuta las siguientes celdas del cuaderno.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [ ]:
driver = webdriver.Chrome()
driver.get("https://webgate.ec.europa.eu/fleet-europa/search_en")

wait = WebDriverWait(driver, 1)

# Hacer clic en el botón "EU"
eu_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//label[@for='countryType1']")))
eu_option.click()

# Hacer click en "All Vessels"
all_vessels_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//label[@for='period1']")))
all_vessels_option.click()

# Hacer clic en el botón "Search"
btn_search = wait.until(EC.element_to_be_clickable((By.XPATH, "//button/span[text()='Search']")))
btn_search.click()


#-----------------------------------------------------------------------------------------------------#

# Esperar a que el selector de la página de resultados se muestre
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-selection--single")))

page_size_selector = driver.find_element(By.CLASS_NAME, "select2-selection--single")
page_size_selector.click()

# Esperar a que la lista con opciones sea visible
wait.until(EC.presence_of_element_located((By.XPATH, "//ul[@class='select2-results__options']")))

# Intentar hacer clic en la opción que contiene el texto "100"
page_size_100 = wait.until(EC.element_to_be_clickable((By.XPATH, "//li[contains(text(),'100')]")))

# Hacer clic en la opción "100"
page_size_100.click()
#-----------------------------------------------------------------------------------------------------#
# Esperar a que la tabla cargue
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table-header-container")))

# Nº de resultados por página 100
page_size_selector = driver.find_element(By.CLASS_NAME, "select2-selection--single")
page_size_selector.click()

# Extraer los nombres de las columnas 
column_headers = driver.find_elements(By.XPATH, "//div[@class='table-header-container']/span")
column_names = [header.text.strip() for header in column_headers] 

# Contenido de las filas
rows = driver.find_elements(By.XPATH, "//table/tbody/tr")

data = []

while True:
    # Extraer filas de la tabla
    rows = driver.find_elements(By.XPATH, "//table/tbody/tr")
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text.strip() for cell in cells]  # Solo tomar los primeros 3 campos
        data.append(row_data)

    # Intentar encontrar el botón "Next"
    try:
        # Buscar el botón "Next" usando el atributo aria-label
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Go to next page']")))

        # Desplazar hasta el botón "Next"
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        
        # Verificar si el botón "Next" está habilitado
        if "disabled" not in next_button.get_attribute("class"):  
            # Hacer clic en el botón "Next"
            next_button.click()
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))  # Esperar a que la nueva página cargue
        else:
            break  # Si el botón está deshabilitado, salir del bucle
    except:
        break  # Si no hay botón "Next" o hubo un error, salir del bucle
   

# Crear DataFrame con todos los datos recopilados
df = pd.DataFrame(data, columns=column_names)

# Mostrar el DataFrame
print(df)

# Cerrar el driver
driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.177)
Stacktrace:
	GetHandleVerifier [0x00007FF6F59F4C25+3179557]
	(No symbol) [0x00007FF6F56588A0]
	(No symbol) [0x00007FF6F54E91CA]
	(No symbol) [0x00007FF6F54C1753]
	(No symbol) [0x00007FF6F556F83E]
	(No symbol) [0x00007FF6F558FBE2]
	(No symbol) [0x00007FF6F5567A03]
	(No symbol) [0x00007FF6F55306D0]
	(No symbol) [0x00007FF6F5531983]
	GetHandleVerifier [0x00007FF6F5A567CD+3579853]
	GetHandleVerifier [0x00007FF6F5A6D1D2+3672530]
	GetHandleVerifier [0x00007FF6F5A62153+3627347]
	GetHandleVerifier [0x00007FF6F57C092A+868650]
	(No symbol) [0x00007FF6F5662FFF]
	(No symbol) [0x00007FF6F565F4A4]
	(No symbol) [0x00007FF6F565F646]
	(No symbol) [0x00007FF6F564EAA9]
	BaseThreadInitThunk [0x00007FF8A1B2259D+29]
	RtlUserThreadStart [0x00007FF8A326AF38+40]


Por otro lado, este utiliza BeautifulSoup para automatizar la extracción de datos desde el sitio web del Registro General de la Flota Pesquera Española. El script automatiza la búsqueda de todos los barcos registrados consultando a través del campo 'CFR' que se encuentra en el dataset 'resultados_fleet.csv', navega a través de las páginas de resultados y guarda los datos en un archivo CSV. Para esto ejecuta las siguientes celdas del cuaderno.

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [16]:
# Cargar el dataset
file_path = r"/Users/macbookairjulio/Documents/GitHub/repo_tipologia/data/resultados_fleet.csv"
df = pd.read_csv(file_path, dtype=str)  # Leer como string para evitar problemas de formato

# Mostrar las primeras filas para entender la estructura
df.head()

,Flag,Vessel Type,CFR,Event Code,Event Date,External Marking,Vessel Name,Ref. Tonnage,Ref. Length,Main Power,IRCS,UVI,Highest Error Level,Reception Timestamp
0,BEL,FX,BEL000021964,DES,27/02/2023,O.2,MIKE MICHEL JR,53.0,21.39,213.0,OPAB,8523448,ERR,01/03/2023 14:47:06
1,BEL,TU,BEL000041982,RET,15/11/1996,BOU 4,ASTRID,15.67,13.85,79.0,OPAD,NaN,NaN,09/09/2019 12:50:19
2,NLD,TU,BEL000041982,MOD,07/06/2024,BR-9,BOURIC,14.0,14.36,83.0,PA2442,NaN,NaN,07/06/2024 23:18:25
3,BEL,FX,BEL000061930,DES,30/08/1996,BOU 6,ANJA,29.79,16.62,103.0,OPAF,NaN,NaN,09/09/2019 12:50:19
4,BEL,TU,BEL000071985,EXP,24/04/2018,BOU.7,DE ENIGE ZOON,57.0,19.1,219.0,OPAG,NaN,MAJ,09/09/2019 12:50:19


In [17]:
# Filtrar solo los buques de España
df_esp = df[df["Flag"] == "ESP"].copy()

# Obtener la cantidad de buques españoles
num_buques_esp = df_esp.shape[0]

# Mostrar las primeras filas después del filtrado
df_esp.describe()

,Flag,Vessel Type,CFR,Event Code,Event Date,External Marking,Vessel Name,Ref. Tonnage,Ref. Length,Main Power,IRCS,UVI,Highest Error Level,Reception Timestamp
count,27484,27484,27484,27484,27484,27484,27484,27484,27484,27484,6875,1116,1563,27484
unique,1,5,27484,7,5556,27476,19706,6055,2750,1933,6286,1116,3,5535
top,ESP,FX,ESP000000001,DES,13/03/2001,3-7PM-1369-91,MARIA,1.5,5.0,0.0,EADI,8799762,MAJ,06/06/2018 02:00:00
freq,27484,27478,1,12469,950,2,150,184,492,2726,4,1,903,13104


In [18]:
# Obtener la lista de identificadores únicos CFR
cfr_list = df_esp["CFR"].dropna().unique().tolist()

# Mostrar los primeros 10 CFR para verificar
cfr_list[:10]

['ESP000000001',
 'ESP000000002',
 'ESP000000003',
 'ESP000000004',
 'ESP000000005',
 'ESP000000006',
 'ESP000000007',
 'ESP000000008',
 'ESP000000009',
 'ESP000000010']

In [ ]:
import requests
from bs4 import BeautifulSoup

# CFR de prueba
cfr = "ESP000000001"
buque_id = "26006"  # El ID que aparece en la URL de detalles

# Construir la URL de detalles del buque
url = f"https://servicio.pesca.mapama.es/CENSO/ConsultaBuqueRegistro/Buques/Details/{buque_id}?foundSearchingText={cfr}&foundInPage=1"

# Hacer la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    print(soup.prettify()) 
else:
    print(f"Error: No se pudo acceder a la página. Código {response.status_code}")


<!DOCTYPE html>
<html lang="es">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="/CENSO/ConsultaBuqueRegistro/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <title>
   Detalles del buque: Código: 1 Nombre: ARGOÑOS
  </title>
  <link href="/CENSO/ConsultaBuqueRegistro/Content/bootstrap/bootstrapf12g24?v=TGfFO1If-ULZb946e3Q565IQRQVoMwtE6A2bSIDxhbM1" rel="stylesheet"/>
  <link href="/CENSO/ConsultaBuqueRegistro/Content/fenixcss?v=z42NHXCPbhKYjIilJT7muAWV6_9v6erQJ2VakY9JTvE1" rel="stylesheet"/>
  <link href="/CENSO/ConsultaBuqueRegistro/Content/css?v=h0rTCPdZqqV6ErwAKJoYgjWvKYV9ApclMeslYMhnwyg1" rel="stylesheet"/>
 </head>
 <body>
  <header class="fenixHeader">
   <img alt="Ministerio de Agricultura, Pesca y Alimentación" class="titulo-logo" src="/CENSO/ConsultaBuqueRegistro/Content/images/iconos/LogoFenixh70.png"/>
   <h1 class="titulo-aplicacion">
    Registro General de la Flota Pesquera
   </h1>
   <spa

In [26]:
import requests
from bs4 import BeautifulSoup

def obtener_detalles_buque(cfr):
    """Busca un buque por CFR y extrae sus características técnicas."""
    
    # 1. Realizar la búsqueda por CFR
    search_url = f"https://servicio.pesca.mapama.es/CENSO/ConsultaBuqueRegistro/Buques/Search?text={cfr}&page=1"
    response = requests.get(search_url)
    if response.status_code != 200:
        return f"Error al acceder a la búsqueda: {response.status_code}"
    
    # 2. Parsear la página de resultados
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el enlace que contiene el CFR buscado
    detalle_link = soup.find('a', href=True, string=cfr)
    if not detalle_link:
        return f"No se encontró el buque con CFR {cfr}"
    
    # 3. Obtener el ID del buque y la URL de detalles
    detalle_url = "https://servicio.pesca.mapama.es" + detalle_link['href']
    response = requests.get(detalle_url)
    if response.status_code != 200:
        return f"Error al acceder a la página de detalles: {response.status_code}"
    
    # 4. Extraer información técnica del buque
    soup = BeautifulSoup(response.text, 'html.parser')
    info_buque = {}
    for row in soup.select("table tr"):
        cols = row.find_all("td")
        if len(cols) == 2:
            key = cols[0].text.strip()
            value = cols[1].text.strip()
            info_buque[key] = value
    
    return info_buque

# Prueba con un CFR específico
cfr_test = "ESP000000001"
datos_buque = obtener_detalles_buque(cfr_test)
print(datos_buque)


No se encontró el buque con CFR ESP000000001
